# Experiment 053: Mixall-Style 4-Model Ensemble with Full Features

**Hypothesis**: The mixall kernel uses a 4-model ensemble (MLP + XGBoost + RF + LightGBM) that may have different CV-LB characteristics than our GP + MLP + LGBM ensemble.

**Key differences from exp_049b (which used Spange only):**
1. Use FULL features: Spange + DRFP + Arrhenius (145 features)
2. Replace GP with XGBoost and RandomForest
3. This is a fundamentally different ensemble composition

**Why this should work:**
- XGBoost and RandomForest have different inductive biases than GP
- The ensemble diversity may improve generalization to unseen solvents
- This may change the CV-LB relationship

In [ ]:
import sys
sys.path.insert(0, '/home/code/experiments/049_manual_ood_handling')

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# Load data
from utils_local import load_data, load_features, generate_leave_one_out_splits, generate_leave_one_ramp_out_splits

print("Loading data...")
X_single_raw, Y_single = load_data("single_solvent")
X_full_raw, Y_full = load_data("full")

print(f"Single solvent: {X_single_raw.shape}, Mixtures: {X_full_raw.shape}")

# Load features
spange = load_features("spange_descriptors")
drfp = load_features("drfps_catechol")
print(f"Spange: {spange.shape}, DRFP: {drfp.shape}")

In [ ]:
# Prepare datasets with FULL features (Spange + DRFP)
def prepare_single_solvent_dataset(X_raw, spange, drfp):
    """Prepare single solvent dataset with all features"""
    solvent_name = X_raw['SOLVENT NAME'].values
    spange_features = spange.loc[solvent_name].values
    drfp_features = drfp.loc[solvent_name].values
    time = X_raw['Residence Time'].values
    temp = X_raw['Temperature'].values
    
    spange_cols = spange.columns.tolist()
    drfp_cols = [f'DRFP_{i}' for i in range(drfp.shape[1])]
    
    df = pd.DataFrame(spange_features, columns=spange_cols)
    df_drfp = pd.DataFrame(drfp_features, columns=drfp_cols)
    df = pd.concat([df, df_drfp], axis=1)
    df['TEMPERATURE'] = temp
    df['TIME'] = time
    df['SOLVENT NAME'] = solvent_name
    
    return df

def prepare_mixture_dataset(X_raw, spange, drfp):
    """Prepare mixture dataset with all features"""
    solvent_a = X_raw['SOLVENT A NAME'].values
    solvent_b = X_raw['SOLVENT B NAME'].values
    solvent_b_pct = X_raw['SolventB%'].values / 100.0
    
    spange_a = spange.loc[solvent_a].values
    spange_b = spange.loc[solvent_b].values
    spange_mix = (1 - solvent_b_pct[:, None]) * spange_a + solvent_b_pct[:, None] * spange_b
    
    drfp_a = drfp.loc[solvent_a].values
    drfp_b = drfp.loc[solvent_b].values
    drfp_mix = (1 - solvent_b_pct[:, None]) * drfp_a + solvent_b_pct[:, None] * drfp_b
    
    solvent_name = [f"{a}.{b}" for a, b in zip(solvent_a, solvent_b)]
    time = X_raw['Residence Time'].values
    temp = X_raw['Temperature'].values
    
    spange_cols = spange.columns.tolist()
    drfp_cols = [f'DRFP_{i}' for i in range(drfp.shape[1])]
    
    df = pd.DataFrame(spange_mix, columns=spange_cols)
    df_drfp = pd.DataFrame(drfp_mix, columns=drfp_cols)
    df = pd.concat([df, df_drfp], axis=1)
    df['TEMPERATURE'] = temp
    df['TIME'] = time
    df['SOLVENT NAME'] = solvent_name
    df['SOLVENT A NAME'] = solvent_a
    df['SOLVENT B NAME'] = solvent_b
    df['SolventB%'] = X_raw['SolventB%'].values
    
    return df

X_single = prepare_single_solvent_dataset(X_single_raw, spange, drfp)
X_mix = prepare_mixture_dataset(X_full_raw, spange, drfp)

print(f"Single solvent dataset: {X_single.shape}")
print(f"Mixture dataset: {X_mix.shape}")

In [ ]:
# Feature extraction with Arrhenius kinetics
def get_spange_features(X_data):
    spange_cols = ['dielectric constant', 'ET(30)', 'alpha', 'beta', 'pi*', 
                   'SA', 'SB', 'SP', 'SdP', 'N', 'n', 'f(n)', 'delta']
    return X_data[spange_cols].values

def get_arrhenius_features(X_data):
    T = X_data['TEMPERATURE'].values
    t = X_data['TIME'].values
    T_kelvin = T + 273.15
    inv_T = 1.0 / T_kelvin
    ln_t = np.log(t + 1e-6)
    interaction = inv_T * ln_t
    return np.column_stack([inv_T, ln_t, interaction, T, t])

def prepare_features(X_data, drfp_mask=None, include_drfp=True):
    """Prepare all features (Spange + DRFP + Arrhenius)"""
    spange = get_spange_features(X_data)
    arrhenius = get_arrhenius_features(X_data)
    
    if include_drfp:
        drfp_cols = [col for col in X_data.columns if col.startswith('DRFP_')]
        drfp_data = X_data[drfp_cols].values
        if drfp_mask is not None:
            drfp_data = drfp_data[:, drfp_mask]
        features = np.hstack([spange, drfp_data, arrhenius])
    else:
        features = np.hstack([spange, arrhenius])
    
    return features

print("Feature extraction functions defined")

In [ ]:
# MLP Model
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dims=[128, 64], dropout=0.2):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 3))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def train_mlp(X_train, Y_train, input_dim, epochs=200, lr=5e-4, weight_decay=1e-4, hidden_dims=[128, 64]):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MLPModel(input_dim, hidden_dims=hidden_dims).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
    criterion = nn.HuberLoss()
    
    X_tensor = torch.FloatTensor(X_train).to(device)
    Y_tensor = torch.FloatTensor(Y_train).to(device)
    
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        pred = model(X_tensor)
        loss = criterion(pred, Y_tensor)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
    
    return model

print("MLP model defined")

In [ ]:
# Mixall-Style 4-Model Ensemble with FULL features
class MixallFullFeaturesModel:
    """
    4-model ensemble: MLP + XGBoost + RandomForest + LightGBM
    Uses FULL features: Spange + DRFP + Arrhenius
    """
    def __init__(self, mlp_weight=0.3, xgb_weight=0.25, rf_weight=0.25, lgbm_weight=0.2):
        self.mlp_weight = mlp_weight
        self.xgb_weight = xgb_weight
        self.rf_weight = rf_weight
        self.lgbm_weight = lgbm_weight
        
        self.scaler = StandardScaler()
        self.mlp_models = []
        self.xgb_models = []
        self.rf_models = []
        self.lgbm_models = []
        
        self.drfp_mask = None
        self.input_dim = None
    
    def fit(self, X_train, Y_train):
        """Train all 4 model types"""
        # Get DRFP mask
        drfp_cols = [col for col in X_train.columns if col.startswith('DRFP_')]
        drfp_data = X_train[drfp_cols].values
        self.drfp_mask = drfp_data.var(axis=0) > 0
        
        # Prepare features
        X_features = prepare_features(X_train, self.drfp_mask, include_drfp=True)
        self.input_dim = X_features.shape[1]
        X_scaled = self.scaler.fit_transform(X_features)
        
        Y_values = Y_train.values
        
        # Train MLP (3 models for bagging)
        for _ in range(3):
            mlp = train_mlp(X_scaled, Y_values, self.input_dim, epochs=200, hidden_dims=[128, 64])
            self.mlp_models.append(mlp)
        
        # Train XGBoost (per-target)
        xgb_params = {
            'objective': 'reg:squarederror',
            'learning_rate': 0.1,
            'max_depth': 6,
            'n_estimators': 200,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'verbosity': 0
        }
        for i in range(3):
            model = xgb.XGBRegressor(**xgb_params)
            model.fit(X_scaled, Y_values[:, i])
            self.xgb_models.append(model)
        
        # Train RandomForest (per-target)
        rf_params = {
            'n_estimators': 200,
            'max_depth': 10,
            'min_samples_split': 5,
            'min_samples_leaf': 2,
            'n_jobs': -1
        }
        for i in range(3):
            model = RandomForestRegressor(**rf_params)
            model.fit(X_scaled, Y_values[:, i])
            self.rf_models.append(model)
        
        # Train LightGBM (per-target)
        lgbm_params = {
            'objective': 'regression',
            'metric': 'mse',
            'learning_rate': 0.03,
            'max_depth': 6,
            'num_leaves': 31,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'verbose': -1,
            'n_estimators': 500
        }
        for i in range(3):
            model = lgb.LGBMRegressor(**lgbm_params)
            model.fit(X_scaled, Y_values[:, i])
            self.lgbm_models.append(model)
        
        return self
    
    def predict(self, X_test):
        """Predict using 4-model ensemble"""
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Prepare features
        X_features = prepare_features(X_test, self.drfp_mask, include_drfp=True)
        X_scaled = self.scaler.transform(X_features)
        
        # MLP predictions
        mlp_preds = []
        X_tensor = torch.FloatTensor(X_scaled).to(device)
        for mlp in self.mlp_models:
            mlp.eval()
            with torch.no_grad():
                pred = mlp(X_tensor).cpu().numpy()
            mlp_preds.append(pred)
        mlp_pred = np.mean(mlp_preds, axis=0)
        
        # XGBoost predictions
        xgb_pred = np.zeros((len(X_test), 3))
        for i, model in enumerate(self.xgb_models):
            xgb_pred[:, i] = model.predict(X_scaled)
        xgb_pred = np.clip(xgb_pred, 0, 1)
        
        # RandomForest predictions
        rf_pred = np.zeros((len(X_test), 3))
        for i, model in enumerate(self.rf_models):
            rf_pred[:, i] = model.predict(X_scaled)
        rf_pred = np.clip(rf_pred, 0, 1)
        
        # LightGBM predictions
        lgbm_pred = np.zeros((len(X_test), 3))
        for i, model in enumerate(self.lgbm_models):
            lgbm_pred[:, i] = model.predict(X_scaled)
        lgbm_pred = np.clip(lgbm_pred, 0, 1)
        
        # Ensemble
        final_pred = (self.mlp_weight * mlp_pred + 
                      self.xgb_weight * xgb_pred + 
                      self.rf_weight * rf_pred + 
                      self.lgbm_weight * lgbm_pred)
        
        return np.clip(final_pred, 0, 1)

print("MixallFullFeaturesModel defined")

In [ ]:
# Run CV for single solvents
print("Running Single Solvent CV with Mixall-Style 4-Model Ensemble...")
print("="*60)

splits = list(generate_leave_one_out_splits(X_single, Y_single))
print(f"Number of folds: {len(splits)}")

solvent_errors = {}
all_preds = []
all_true = []

for fold_idx, (train_idx, test_idx) in enumerate(splits):
    X_train = X_single.iloc[train_idx]
    Y_train = Y_single.iloc[train_idx]
    X_test = X_single.iloc[test_idx]
    Y_test = Y_single.iloc[test_idx]
    
    test_solvent = X_test['SOLVENT NAME'].iloc[0]
    
    # Train model
    model = MixallFullFeaturesModel()
    model.fit(X_train, Y_train)
    
    # Predict
    preds = model.predict(X_test)
    
    # Calculate MSE
    mse = np.mean((preds - Y_test.values) ** 2)
    solvent_errors[test_solvent] = mse
    
    all_preds.append(preds)
    all_true.append(Y_test.values)
    
    print(f"Fold {fold_idx+1:2d}: {test_solvent:45s} MSE = {mse:.6f}")

all_preds = np.vstack(all_preds)
all_true = np.vstack(all_true)
single_mse = np.mean((all_preds - all_true) ** 2)
single_std = np.std([solvent_errors[s] for s in solvent_errors])

print(f"\nMixall-Style Single Solvent CV MSE: {single_mse:.6f} +/- {single_std:.6f}")

In [ ]:
# Run CV for mixtures
print("\n" + "="*60)
print("Running Mixture CV with Mixall-Style 4-Model Ensemble...")
print("="*60)

mix_splits = list(generate_leave_one_ramp_out_splits(X_mix, Y_full))
print(f"Number of folds: {len(mix_splits)}")

mix_errors = {}
mix_preds = []
mix_true = []

for fold_idx, (train_idx, test_idx) in enumerate(mix_splits):
    X_train = X_mix.iloc[train_idx]
    Y_train = Y_full.iloc[train_idx]
    X_test = X_mix.iloc[test_idx]
    Y_test = Y_full.iloc[test_idx]
    
    test_mixture = X_test['SOLVENT NAME'].iloc[0]
    
    # Train model
    model = MixallFullFeaturesModel()
    model.fit(X_train, Y_train)
    
    # Predict
    preds = model.predict(X_test)
    
    # Calculate MSE
    mse = np.mean((preds - Y_test.values) ** 2)
    mix_errors[test_mixture] = mse
    
    mix_preds.append(preds)
    mix_true.append(Y_test.values)
    
    print(f"Fold {fold_idx+1:2d}: {test_mixture:55s} MSE = {mse:.6f}")

mix_preds = np.vstack(mix_preds)
mix_true = np.vstack(mix_true)
mix_mse = np.mean((mix_preds - mix_true) ** 2)
mix_std = np.std([mix_errors[s] for s in mix_errors])

print(f"\nMixall-Style Mixture CV MSE: {mix_mse:.6f} +/- {mix_std:.6f}")

In [ ]:
# Calculate overall CV score
print("\n" + "="*60)
print("Mixall-Style 4-Model Ensemble Overall Results")
print("="*60)

n_single = len(all_true)
n_mix = len(mix_true)
n_total = n_single + n_mix

overall_mse = (n_single * single_mse + n_mix * mix_mse) / n_total

print(f"\nSingle Solvent CV MSE: {single_mse:.6f} +/- {single_std:.6f} (n={n_single})")
print(f"Mixture CV MSE: {mix_mse:.6f} +/- {mix_std:.6f} (n={n_mix})")
print(f"Overall CV MSE: {overall_mse:.6f}")

print(f"\nBaseline (exp_030): CV = 0.008298")
print(f"exp_049b (Mixall with Spange only): CV = 0.014196")
print(f"Improvement vs baseline: {(0.008298 - overall_mse) / 0.008298 * 100:.1f}%")
print(f"Improvement vs exp_049b: {(0.014196 - overall_mse) / 0.014196 * 100:.1f}%")

if overall_mse < 0.008298:
    print("\n✓ BETTER than baseline!")
else:
    print("\n✗ WORSE than baseline.")

In [ ]:
# Final Summary
print("\n" + "="*60)
print("EXPERIMENT 053 SUMMARY")
print("="*60)

print(f"\nMixall-Style 4-Model Ensemble (MLP + XGBoost + RF + LightGBM):")
print(f"  Features: Spange + DRFP + Arrhenius (full features)")
print(f"  Single Solvent CV: {single_mse:.6f}")
print(f"  Mixture CV: {mix_mse:.6f}")
print(f"  Overall CV: {overall_mse:.6f}")
print(f"  vs Baseline (exp_030): {(overall_mse - 0.008298) / 0.008298 * 100:+.1f}%")

print("\nKey insights:")
print("1. 4-model ensemble: MLP + XGBoost + RF + LightGBM")
print("2. Uses FULL features (Spange + DRFP + Arrhenius)")
print("3. Different from exp_049b which used Spange only")

if overall_mse < 0.008298:
    print("\nCONCLUSION: Mixall-style ensemble with full features IMPROVES CV!")
    print("This is a fundamentally different approach that may change the CV-LB relationship.")
else:
    print("\nCONCLUSION: Mixall-style ensemble with full features does NOT improve CV.")
    print("The 4-model ensemble doesn't help with the current features.")

print(f"\nRemaining submissions: 5")
print(f"Best model: exp_030 (GP 0.15 + MLP 0.55 + LGBM 0.3) with CV 0.008298, LB 0.0877")